# Ansible is


 - configuration manager 
 - simple 
 - extensible via modules
 - written in python
 - broad community
 - many external tools
 - playbook repository
 - used by openstack & tonns of project
 
 


 # Configuration Manager
 
Explain infrastructure as code (e.g., remember `docker-compose`?):

- managing data centers through machine-readable definition files;
- install, configure, decommission physical and virtual resources;
- avoid manual processes;
- speed ++, costs --, risk -- (see [Google SRE Book](https://sre.google/workbook/postmortem-culture/) );
- declarative (describe the expected state) or imperative (describe the steps) approaches;
- push or pull model;

[![](https://mermaid.ink/img/pako:eNrNVMFu2zAM_RVBhyEB4sB2Yjfxhh7W9RB0RYclp0U9KDaTGLElT5LbuXH-fZQVdw3QATvuYpP0I_keKetIU5kBTajneUyY3BSQkIXYKq6NqlNTKyBckxsEMdFhdopXe_L1OxNMZPA0GDD6QWx09dE9tzzZcq_WoIgzC14ZWb2FsC7xbNPh0Ba-Jq2DWzpeVReFp-BnDdog2rrk7LZoVHI9-INGgA3p3EjVoKNTlVdGo2WgrApuwNp7qY0ePrpeIHa5AKvAWQSVkfYaYQ93Lckv5HdkLaD9hN_vHto-uU8de7LdgSEZN7wli9Xyfm0fZ_k4iKfXWeyAK40cdL1xY3Q11gt-Q25duUsgF7xodK672ZMnXuTYJZfimO4hPZxcOIOqkE0JwlhSmZX12uBSy_pys9ggyxWktiJZfcZBK5mtGf2Gr7qLouJ3ZLzj9Dg7hA3XvY6_uD1agHmW6uA9I4uM_GOM0Ue7Wm2Q6codkP-T43kXRS4OS9MUQCYERy8PkFjEm3jYx3cKADPoiJagSp5n-GMemSCEUbOHEhhN0Mxgy-vCMMrECaF1hZThNrPnnya4WxhRXhu5bETa-w7zJed4KkqKdAuN0YqLH1KWPQhdmhzpL5oE0-l46kdR4E_DyWzuz0e0oUk4j8ZxNI8nYRyH4Sy4Oo3oS5fvj-MgCv0oiGdX4dSP_WBEoSN0726X7pI5_QYwkpLJ?type=png)](https://mermaid.live/edit#pako:eNrNVMFu2zAM_RVBhyEB4sB2Yjfxhh7W9RB0RYclp0U9KDaTGLElT5LbuXH-fZQVdw3QATvuYpP0I_keKetIU5kBTajneUyY3BSQkIXYKq6NqlNTKyBckxsEMdFhdopXe_L1OxNMZPA0GDD6QWx09dE9tzzZcq_WoIgzC14ZWb2FsC7xbNPh0Ba-Jq2DWzpeVReFp-BnDdog2rrk7LZoVHI9-INGgA3p3EjVoKNTlVdGo2WgrApuwNp7qY0ePrpeIHa5AKvAWQSVkfYaYQ93Lckv5HdkLaD9hN_vHto-uU8de7LdgSEZN7wli9Xyfm0fZ_k4iKfXWeyAK40cdL1xY3Q11gt-Q25duUsgF7xodK672ZMnXuTYJZfimO4hPZxcOIOqkE0JwlhSmZX12uBSy_pys9ggyxWktiJZfcZBK5mtGf2Gr7qLouJ3ZLzj9Dg7hA3XvY6_uD1agHmW6uA9I4uM_GOM0Ue7Wm2Q6codkP-T43kXRS4OS9MUQCYERy8PkFjEm3jYx3cKADPoiJagSp5n-GMemSCEUbOHEhhN0Mxgy-vCMMrECaF1hZThNrPnnya4WxhRXhu5bETa-w7zJed4KkqKdAuN0YqLH1KWPQhdmhzpL5oE0-l46kdR4E_DyWzuz0e0oUk4j8ZxNI8nYRyH4Sy4Oo3oS5fvj-MgCv0oiGdX4dSP_WBEoSN0726X7pI5_QYwkpLJ)



Trick: use a light development environment based on the introductory session.

[![](https://mermaid.ink/img/pako:eNp1Uk1v2zAM_SsCD0MCxEVqp67jFru0OwxoL91tVjEoNpMYtSVNH0HdJP99lK1g62EwRJDy4-MjqSPUqkEoIUkSLl3rOizZd7k1wjrja-cNMmHZA4G4HDE7I_SePb1wGb4GD7MZhy9yY_XdZLei3IrEWzRscjuhndL_QviYGH2YzwOT9ZuJ2qBW1QsZ2zplhkgSZJK7Cf6uda_E0BqsXavkKCZk_apmuhPDRqk3OydErxrfoa1m0bnk68HtlQwIlE3sInT3lZ3-Fku077rE4G-P1pHgELIYnkaRUwrKXSvxUwdEh_JQPeKB_h4-TYFq1qqn3rB6UNIJSjWW2KOyRtVv49z-G71GtVF5JGO0mtP9KVRmXMICejS9aBta7JFLxji4PfbIoSS3wa3wnePA5ZmgXjfC4bcmTBtK2jouQHinfgyyvsQT5rEV1GAPpKazdKuF_KlUfwFRCOUR3qG8zourPF2u8tt0nRbrfFUsYKDroriis86DSW-y2-y8gI-RYUn4LE2z6_Rmna3y5ZIycJT0PL3P8Zme_wCRge2O?type=png)](https://mermaid.live/edit#pako:eNp1Uk1v2zAM_SsCD0MCxEVqp67jFru0OwxoL91tVjEoNpMYtSVNH0HdJP99lK1g62EwRJDy4-MjqSPUqkEoIUkSLl3rOizZd7k1wjrja-cNMmHZA4G4HDE7I_SePb1wGb4GD7MZhy9yY_XdZLei3IrEWzRscjuhndL_QviYGH2YzwOT9ZuJ2qBW1QsZ2zplhkgSZJK7Cf6uda_E0BqsXavkKCZk_apmuhPDRqk3OydErxrfoa1m0bnk68HtlQwIlE3sInT3lZ3-Fku077rE4G-P1pHgELIYnkaRUwrKXSvxUwdEh_JQPeKB_h4-TYFq1qqn3rB6UNIJSjWW2KOyRtVv49z-G71GtVF5JGO0mtP9KVRmXMICejS9aBta7JFLxji4PfbIoSS3wa3wnePA5ZmgXjfC4bcmTBtK2jouQHinfgyyvsQT5rEV1GAPpKazdKuF_KlUfwFRCOUR3qG8zourPF2u8tt0nRbrfFUsYKDroriis86DSW-y2-y8gI-RYUn4LE2z6_Rmna3y5ZIycJT0PL3P8Zme_wCRge2O)

Ansible uses the push model and supports both declarative and imperative approaches.

 # Advantages
 
 No agents: ansible copies python and all deployment scripts/modules to the target machine via ssh and executes them remotely. Some modules though require that target hosts contain specific python libraries.
 
 Jobs are executed in parallel, but you can configure for serialization using different strategies for speed up, rollout or other purposes: (link)
  
 Authentication can be passwordless (ssh/pki, kerberos) or with password.
 
 Automation jobs (Playbooks) are described via YAML - a very concise and simple language.
 You can validate and lint files with yamllint and [ansible-lint](https://github.com/ansible/ansible-lint).
 
```yaml
this_is:
  a: yaml

file:
- with dict
- a list
```
 
Passwords are supported, but SSH keys with ssh-agent are one of the best ways to use Ansible. Though if you want to use Kerberos, that's good too. 
 
You have a lot of options! Root logins are not required, you can login as any user, and then su or sudo to any user.

In [ ]:
cd /notebooks/exercise-00/

In [ ]:
# Let's check our ansible directory
!tree

## ansible.cfg 

It's the main configuration file. While all ansible are in YAML, `ansible.cfg` is in .ini format. Eg.

```
# This is a .ini file.
[stanza]
key = value
```

Let's check the content of a sample ansible.cfg:

  - there's a lot of stuff in there
  - there will be more ;)
  - for now **let's check only the uncommented ones**.

In [ ]:
!cat ansible.cfg

## Inventories

A simple inventory file contains a static list of nodes to contact.

Generally, an [inventory can be static or dynamic](https://docs.ansible.com/ansible/latest/inventory_guide/intro_inventory.html), as we will see in the following lessons.



In [ ]:
!cat inventory

In [ ]:
# You can have many inventory files
!cat staging

## Environment variables

**N.B. ansible environment variables are not related with process environment**

#### Exercise:

- using [the terminal](/terminals/ps) identify the process executing this notebook by the current working directory `/proc/*/cwd`;
- check the environment variables of the process with `cat /proc/<pid>/environ | od -a`;

You defined your host groups in the environment, eg:

  - course
  - ansible
  - staging
  
Ansible defines [two default groups: all and ungrouped](http://docs.ansible.com/ansible/latest/intro_inventory.html#default-groups).

You can assign variables to all hosts using the `all` group.

In [ ]:
# group_vars - a directory containing environment files for various host groups.
!tree group_vars

In [ ]:
# I set  env_name in two different files
!grep env_name -r group_vars/
!cat group_vars/staging

In [ ]:
# The debug module (-m debug) shows variables' content or dumps messages.
#   by default uses the inventory set into ansible.cfg, thus writing
!ansible all -m debug -a 'var=env_name'

#### Exercise

Dump env_name tied to the staging inventory.

  - which is the expected output?
  - what ties the "staging" inventory file to group_vars/staging?


In [ ]:
# Solution
!ansible all -i staging -m debug -a 'var=env_name'

In [ ]:
# Use this cell for the exercise

#### Exercise

- Read the inventory and try to predict the output of the following command
- Rename the `staging` inventory file to `foobar`. What happens?
- Restore the original name.

In [ ]:
#
# Read the inventory and try to predict the output of
#
!ansible course -i staging -m debug -a 'var=proxy_env'